In [1]:
from collections import defaultdict, deque
import random
import math
import time

import numpy as np

class City:
    id = 0
    def __init__(self):
        self.X = random.randint(-100, 100)
        self.Y = random.randint(-100, 100)
        self.Z = random.randint(0, 50)
        self.coordinates = np.array((self.X, self.Y, self.Z))
        self.id = City.id
        City.id += 1
    
class Graph:
    
    def __init__(self, cities):
        self.all_paths = []
        self.distance_list = []
        self.adjList = defaultdict(list)
        self.sym_matrix = self.generate_sym_matrix(cities)
        self.matrix_80 = self.generate_80_matrix(cities)
        self.asym100_matrix = self.generate_asym_matrix(cities)
        self.asym80_matrix = self.generate_80asym_matrix(cities)

    #Stworzenie symetrycznej macierzy 100%

    def generate_sym_matrix(self, cities):
        sym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            sym_matrix.append(row)
        return np.array(sym_matrix)
    

    #Stworzenie symetrycznej macierzy 80%

    def generate_80_matrix(self, cities):
        matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i != j:
                    row.append(1)
                else:
                    row.append(0)
            matrix.append(row)
        matrix = np.array(matrix)
        total_elements = matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(matrix.shape)
    

    #Stworzenie asymetrycznej macierzy 100%

    def generate_asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        return np.array(asym_matrix)
    

    #Stworzenie asymetrycznej macierzy 80%

    def generate_80asym_matrix(self, cities):
        asym_matrix = []
        for i in range(0, len(cities) - 1):
            row = []
            for j in range(0, len(cities) - 1):
                if i == j:
                    row.append(0)
                elif cities[i].Z < cities[j].Z and i != j:
                    row.append(2)
                elif cities[i].Z >= cities[j].Z and i != j:
                    row.append(1)
            asym_matrix.append(row)
        asym_matrix = np.array(asym_matrix)
        total_elements = asym_matrix.size
        num_to_change = int(0.2 * total_elements)
    
        indices_to_change = np.random.choice(total_elements, num_to_change, replace=False)
        matrix_flattened = asym_matrix.flatten()
    
        for index in indices_to_change:
            if matrix_flattened[index] == 1 or matrix_flattened[index] == 2:
                matrix_flattened[index] = 0
    
        return matrix_flattened.reshape(asym_matrix.shape)


    #Metoda obliczajaca odleglosc bez uwzglednienia asymetrycznosci
    def calculate_distance(self, city1, city2):
        return np.linalg.norm(city1.coordinates - city2.coordinates)
    


    #Metoda obliczajaca odleglosc z uwzglednieniem asymetrycznosci
    def calculate_distance_asym(self, city1, city2):
        if city1.Z < city2.Z:
            #print("Going up")
            return 1.1 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        elif city1.Z > city2.Z:
            #print("Going down")
            return 0.9 * math.sqrt((city2.X - city1.X) ** 2 + (city2.Y - city1.Y) ** 2 + (city2.Z - city1.Z) ** 2)
        else:
            #print("Same height")
            return np.linalg.norm(city1.coordinates - city2.coordinates)


    #Dodawanie krawedzi do grafu (do zmiennej adjList)
    def addEdge(self, u, v):
            self.adjList[u].append(v)

    #Definiowanie krawedzi i przekazanie do metody addEdge
    def createEdge(self, matrix):
        for i in range(0, len(matrix)):
            for j in range(0, len(matrix)):
                if matrix[i][j] == 1 or matrix[i][j] == 2:
                    self.addEdge(i, j)
                    #print(f"Created edge between {i} and {j}")

                    
    #Metoda obliczajaca wszystkie mozliwe sciezki metoda BFS
    def bfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.popleft()
            #print(f"BFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    queue.append((neighbour, path + [neighbour]))
        return paths
    
    #Metoda obliczajaca wszystkie mozliwe sciezki metoda DFS
    def dfs(self, startNode):
        queue = deque([(startNode, [startNode])])
        paths = []

        while queue:
            currentNode, path = queue.pop()
            #print(f"DFS: {path}")
            
            if len(path) == len(self.adjList) + 1:
                paths.append(path)
                print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    queue.append((neighbour, path + [neighbour]))
                elif len(path) == len(self.adjList) and neighbour == 0:
                    # print(f'{len(path)} == {len(self.adjList)}')
                    # print(f'AdjList: {self.adjList}')
                    queue.append((neighbour, path + [neighbour]))
        return paths
    
    #Wersja dla 1 NN
    def NN(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        visited.add(startNode)
        distance = 0
        currentNode = startNode

        while len(path) < len(self.adjList) + 1:
            closest_distance = float('inf')
            closest_neighbor = None
            
            for neighbour in self.adjList[currentNode]:
                if neighbour not in visited:
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])

                    if dist < closest_distance:
                        closest_distance = dist
                        closest_neighbor = neighbour


                elif len(path) == len(self.adjList) and neighbour == 0:
                    path.append(neighbour)
                    if sym == 'sym':
                        dist = self.calculate_distance(cities[currentNode], cities[neighbour])
                    elif sym == 'asym':
                        dist = self.calculate_distance_asym(cities[currentNode], cities[neighbour])
                    distance += dist
                    continue
            
            if closest_neighbor is not None:
                path.append(closest_neighbor)
                distance += closest_distance
                visited.add(closest_neighbor)
                currentNode = closest_neighbor
            else:
                print(f'{currentNode} has no neighoburs')
                break
        return path, distance

    def NN2(self, startNode, sym, cities):
        path = [startNode]
        visited = set()
        dist = []
        visited.add(startNode)
        distance = 0
        currentNode = []
        neighbours = []
        currentNode.append(startNode)

        while len(path) < len(self.adjList) + 1:
            #print(f'Path len:{len(path)}, {len(self.adjList)}')
            closest_distance = float('inf')
            closest_neighbor = None
            
            for i in range(0, len(currentNode)):
                print(f'\033[91mCurrent node: {currentNode}, {i}, Len: {len(currentNode)}\033[0m')
                for neighbour in self.adjList[currentNode[i]]:
                    if neighbour not in visited:
                        if sym == 'sym':
                            dist.append([self.calculate_distance(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        elif sym == 'asym':
                            dist.append([self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]), neighbour])
                            neighbours.append(neighbour)
                        print(f'added: {neighbour}')


                    elif len(path) == len(self.adjList) and neighbour == 0:
                        print(f'Dodatkowe, {path}')
                        path.append(neighbour)
                        if sym == 'sym':
                            dist.append(self.calculate_distance(cities[currentNode[i]], cities[neighbour]))
                        elif sym == 'asym':
                            dist.append(self.calculate_distance_asym(cities[currentNode[i]], cities[neighbour]))
                        distance = np.float64(distance)
                        distance += dist
                        break
                    print(f'\033[92mNieghbours: {neighbours}\033[0m')
                    



                if len(neighbours) == 0 and len(path) < len(self.adjList):
                    print(f'Bad path: {path}')
                    print(len(path), len(self.adjList), len(path) != len(self.adjList))
                    print(f'Error, {self.adjList[currentNode[i]]}')
                    path.remove(currentNode[0])
                    visited.remove(currentNode[0])
                    path.append(currentNode[1])
                    visited.add(currentNode[1])
                    print(f'Path: {path}, visited: {visited}')
                neighbours.clear()

                if len(dist) != 0:
                    print(f'is not None')
                    break

            if len(path) < len(self.adjList):
                print(dist, type(dist))
                dist = sorted(dist, key=lambda row: row[0])
                print(f'Distance: {dist}')
                closest_distance = dist[0][0]
                # if len(path) < len(self.adjList) - 1:
                #     closest_neighbor = [dist[0][1], dist[1][1]]
                if len(dist) >= 2:
                    closest_neighbor = [dist[0][1], dist[1][1]]
                else:
                    closest_neighbor = [dist[0][1]]
                dist.clear()
            
            if closest_neighbor is not None:
                path.append(closest_neighbor[0])
                print(f'Path: {path}')
                distance += closest_distance
                visited.add(closest_neighbor[0])
                currentNode = closest_neighbor
            else:
                print(f'{currentNode} has no neighoburs')
                break
        return path, distance



    
    #Metoda obliczajaca najkrotsza droge uwzgledniajac jedynie symetryczne grafy oraz metode wyszukiwania
    def find_shortest_sym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        paths = []
        if method == 'bfs':
            print('bfs')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('dfs')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {path}'
    

    #Metoda obliczajaca najkrotsza droge uwzgledniajac jedynie asymetryczne grafy oraz metode wyszukiwania
    def find_shortest_asym_path(self, cities, method):
        min_distance = float('inf')
        path = float('inf')
        distance = 0
        paths = []
        if method == 'bfs':
            print('bfs')
            paths = self.bfs(0)
        elif method == 'dfs':
            print('dfs')
            paths = self.dfs(0)
        #print(f"Paths: {paths}\n\n")
        for i in range(0, len(paths)):
            #print(f"{i}: {paths[i]}")
            for j in range(0, len(paths[i]) - 1):
                #print(f"({paths[i][j]}{paths[i][j + 1]})")
                distance += self.calculate_distance_asym(cities[paths[i][j]], cities[paths[i][j+1]])
            #print(f"Distance in path {paths[i]}: {distance}")
            if distance < min_distance:
                min_distance = distance
                path = i + 1
            distance = 0
        return f'Min distance is: {min_distance} for path {path}'
    
    #Metoda obliczajaca heurystyki
    def heuristic(self, cities):
        h_dop = float('inf') #Min odlegosc
        h_niedop = 0 #Srednia odleglosc
    
        current_dist = 0
        divide = 0
        for i in range(len(cities)):
            for j in self.adjList[i]:

                current_dist = self.calculate_distance(cities[i], cities[j])
                h_niedop += current_dist
                # print(f'Current distance between {i} and {j}: {current_dist}')

                divide += 1

                if current_dist < h_dop:
                    # print(f'{current_dist} is lower than {h_dop}')
                    h_dop = current_dist
        
        h_niedop = h_niedop / divide
        # print(f'h_niedop = {h_niedop}, h_dop = {h_dop}')
        return h_dop, h_niedop
                
    #Metoda obliczajaca najkrotsza trase metoda A*
    def astar(self, startNode, cities):
        h_dop, h_niedop = self.heuristic(cities)
        queue = deque([(0, startNode, [startNode], 0)])
        paths = []
        h_chosen = h_dop
        total_distance = 0
        print(h_chosen)

        while queue:
            # print(f'\nBefore: {queue}')
            queue = deque(sorted(queue, key=lambda row: row[0]))
            # print(f'After: {queue}')
            total_cost, currentNode, path, distance_passed = queue.popleft()
            # print(f'Popped cost: {total_cost}, path: {path}\n')

            # print(path)
            
            if len(path) == len(self.adjList):
                paths.append(path)
                print(path)
                continue

            for neighbour in self.adjList[currentNode]:
                if neighbour not in path:
                    actual_cost = self.calculate_distance(cities[currentNode], cities[neighbour])
                    g_n = distance_passed + actual_cost
                    h_n = (6-len(path)) * h_chosen
                    # print(f'Heuristic from {neighbour}: {h_n}, G_n cost: {g_n}, toal cost: {total_cost}, actual cost: {actual_cost}')
                    f_cost = g_n + h_n
                    queue.append((f_cost, neighbour, path + [neighbour], g_n))
                elif len(path) == len(self.adjList) - 1 and neighbour == 0:
                    print('=============================================')
                    path.append(0)
                    paths.append(path)
                    print(path)
                    queue.clear()
        paths = paths[0]
        for i in range(0, len(paths) - 1):
            total_distance += self.calculate_distance(cities[paths[i]], cities[paths[i + 1]])
        return paths, total_distance

                


In [4]:
cities = []
for i in range(0, 6):
    cities.append(City())
    print(f'City {i}: X: {cities[i].X}, Y: {cities[i].Y}, Z: {cities[i].Z}')
graph = Graph(cities)

#print(graph.sym_matrix)
#print(graph.matrix_80)
# print(graph.asym100_matrix)
#print(graph.asym80_matrix)

#graph.createEdge(graph.asym100_matrix)
graph.createEdge(graph.sym_matrix)
#graph.createEdge(graph.matrix_80)
#graph.createEdge(graph.asym80_matrix)

start_time = time.perf_counter()
min_distance_bfs = graph.find_shortest_sym_path(cities, 'dfs')
print(min_distance_bfs)
end_time = time.perf_counter()
print(f'DFS time: {end_time - start_time}')


# start_time = time.perf_counter()
# min_distance_bfs = graph.find_shortest_asym_path(cities, 'bfs') 
# print(min_distance_bfs)
# end_time = time.perf_counter()
# print(f'DFS time: {end_time - start_time}')

start_time = time.perf_counter()
NN_path, NN_distance = graph.NN2(0, 'sym', cities)
print(f'Path: {NN_path}, distance: {NN_distance}')
end_time = time.perf_counter()
print(f'NN time: {end_time - start_time}')


start_time = time.perf_counter()
best_path, best_distance = graph.astar(0, cities)
print(f'Best path: {best_path}, best dist: {best_distance}')
end_time = time.perf_counter()
print(f'A* time: {end_time - start_time}')


City 0: X: -72, Y: 86, Z: 19
City 1: X: -71, Y: -74, Z: 36
City 2: X: -59, Y: 82, Z: 35
City 3: X: 43, Y: 12, Z: 16
City 4: X: 16, Y: -57, Z: 24
City 5: X: 56, Y: -8, Z: 13
dfs
[0, 4, 3, 2, 1, 0]
[0, 4, 3, 1, 2, 0]
[0, 4, 2, 3, 1, 0]
[0, 4, 2, 1, 3, 0]
[0, 4, 1, 3, 2, 0]
[0, 4, 1, 2, 3, 0]
[0, 3, 4, 2, 1, 0]
[0, 3, 4, 1, 2, 0]
[0, 3, 2, 4, 1, 0]
[0, 3, 2, 1, 4, 0]
[0, 3, 1, 4, 2, 0]
[0, 3, 1, 2, 4, 0]
[0, 2, 4, 3, 1, 0]
[0, 2, 4, 1, 3, 0]
[0, 2, 3, 4, 1, 0]
[0, 2, 3, 1, 4, 0]
[0, 2, 1, 4, 3, 0]
[0, 2, 1, 3, 4, 0]
[0, 1, 4, 3, 2, 0]
[0, 1, 4, 2, 3, 0]
[0, 1, 3, 4, 2, 0]
[0, 1, 3, 2, 4, 0]
[0, 1, 2, 4, 3, 0]
[0, 1, 2, 3, 4, 0]
Min distance is: 471.0426579141475 for path 15
DFS time: 0.0008074999786913395
Current node: [0], 0, Len: 1
added: 1
Nieghbours: [1]
added: 2
Nieghbours: [1, 2]
added: 3
Nieghbours: [1, 2, 3]
added: 4
Nieghbours: [1, 2, 3, 4]
is not None
[[160.90369790654285, 1], [21.0, 2], [136.7845020461017, 3], [167.98214190800164, 4]] <class 'list'>
Distance: [[21.0, 2], [136.7

1
